### This is a model made to recommend github repositories - in experimentation phase

The ideal would be to have a dataset with user interactions. My plan was the following:

* Combining both our current repository dataset with a user interaction dataset. 
* From these two datasets, we can compute a cosine similarity score between repositories and users.
* The higher the score, the more likely the user will like the repository.
* So we can then use this score to recommend repositories to users.

To be researched. The following is a working structure to train the above model, yet at the moment it is using a dummy target variable. If a user interaction dataset is found or created, the model can be trained with it.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch

In [2]:
# load the data
df = pd.read_csv('../Data/clean/allReposCleaned.csv')
df

,id,name,owner_user,owner_type,description,url,is_fork,date_created,date_updated,date_pushed,...,has_wiki,has_pages,has _discussions,num_forks,is_archived,is_disabled,is_template,license,open_issues,topics
0,1618628,beforerender,spejman,User,Add hook like afterfilter runs action rendered,https://api.github.com/repos/spejman/before_re...,True,2011-04-15T11:30:07Z,2013-01-01T21:48:40Z,2010-09-06T03:36:34Z,...,True,False,False,0,False,False,False,MIT License,0,[]
1,298819,bliptv,spejman,User,Ruby library Bliptv API,https://api.github.com/repos/spejman/bliptv,True,2009-09-06T01:07:24Z,2012-12-13T01:05:54Z,2009-09-09T08:06:39Z,...,True,False,False,1,False,False,False,MIT License,0,[]
2,26987,bn4r,spejman,User,Bayesian networks Ruby,https://api.github.com/repos/spejman/bn4r,False,2008-06-20T09:54:42Z,2019-08-13T13:27:12Z,2012-07-22T19:10:22Z,...,True,True,False,6,False,False,False,No license,0,[]
3,1590903,cachemoney,spejman,User,WriteThrough Cacheing Library ActiveRecord,https://api.github.com/repos/spejman/cache-money,True,2011-04-09T09:33:31Z,2013-01-01T20:33:32Z,2011-03-16T22:45:43Z,...,True,False,False,0,False,False,False,Apache License 2.0,0,[]
4,276899,contacts,spejman,User,universal interface import email contacts prov...,https://api.github.com/repos/spejman/contacts,True,2009-08-13T11:16:05Z,2012-12-13T00:14:44Z,2009-08-07T19:09:11Z,...,True,False,False,1,False,False,False,"BSD 3-Clause ""New"" or ""Revised"" License",0,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56255,95652496,AwesomeARKit,olucurious,User,curated list awesome ARKit projects resources ...,https://api.github.com/repos/olucurious/Awesom...,False,2017-06-28T09:29:01Z,2024-02-07T07:37:51Z,2023-12-05T08:25:07Z,...,True,False,False,929,False,False,False,No license,4,"['arkit', 'augmented-reality', 'ios', 'objecti..."
56256,32012751,Nuke,kean,User,Image loading system,https://api.github.com/repos/kean/Nuke,False,2015-03-11T10:53:26Z,2024-02-08T14:13:04Z,2024-02-08T17:33:32Z,...,False,False,True,507,False,False,False,MIT License,19,"['cache', 'gif', 'image', 'progressive', 'swif..."
56257,28463126,KeychainAccess,kishikawakatsumi,User,Simple Swift wrapper Keychain works iOS watchO...,https://api.github.com/repos/kishikawakatsumi/...,False,2014-12-24T22:04:11Z,2024-02-09T09:06:03Z,2024-01-12T10:11:48Z,...,False,False,True,790,False,False,False,MIT License,53,"['keychain', 'security', 'touch-id']"
56258,150961965,Dozer,Mortennn,User,Hide menu bar icons macOS,https://api.github.com/repos/Mortennn/Dozer,False,2018-09-30T11:37:51Z,2024-02-09T16:36:35Z,2023-11-30T13:45:27Z,...,True,False,False,251,False,False,False,Mozilla Public License 2.0,86,"['macos', 'minimalistic', 'statusbar', 'utility']"


In [3]:
# delete columns that are not needed
df = df.drop(columns=[ 'id', 'is_archived', 'is_disabled', 'is_template', 'has_projects', 
        'has _discussions', 'owner_type', 'has_pages', 'has_wiki', 
        'has_issues', 'has_downloads', 'is_fork'])
df

,name,owner_user,description,url,date_created,date_updated,date_pushed,size,stars,watchers,updated_at,language,num_forks,license,open_issues,topics
0,beforerender,spejman,Add hook like afterfilter runs action rendered,https://api.github.com/repos/spejman/before_re...,2011-04-15T11:30:07Z,2013-01-01T21:48:40Z,2010-09-06T03:36:34Z,84,1,1,2013-01-01T21:48:40Z,Ruby,0,MIT License,0,[]
1,bliptv,spejman,Ruby library Bliptv API,https://api.github.com/repos/spejman/bliptv,2009-09-06T01:07:24Z,2012-12-13T01:05:54Z,2009-09-09T08:06:39Z,131,1,1,2012-12-13T01:05:54Z,Ruby,1,MIT License,0,[]
2,bn4r,spejman,Bayesian networks Ruby,https://api.github.com/repos/spejman/bn4r,2008-06-20T09:54:42Z,2019-08-13T13:27:12Z,2012-07-22T19:10:22Z,180,19,19,2019-08-13T13:27:12Z,Ruby,6,No license,0,[]
3,cachemoney,spejman,WriteThrough Cacheing Library ActiveRecord,https://api.github.com/repos/spejman/cache-money,2011-04-09T09:33:31Z,2013-01-01T20:33:32Z,2011-03-16T22:45:43Z,874,1,1,2013-01-01T20:33:32Z,Ruby,0,Apache License 2.0,0,[]
4,contacts,spejman,universal interface import email contacts prov...,https://api.github.com/repos/spejman/contacts,2009-08-13T11:16:05Z,2012-12-13T00:14:44Z,2009-08-07T19:09:11Z,106,1,1,2012-12-13T00:14:44Z,Ruby,1,"BSD 3-Clause ""New"" or ""Revised"" License",0,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56255,AwesomeARKit,olucurious,curated list awesome ARKit projects resources ...,https://api.github.com/repos/olucurious/Awesom...,2017-06-28T09:29:01Z,2024-02-07T07:37:51Z,2023-12-05T08:25:07Z,1092,7758,7758,2024-02-07T07:37:51Z,Swift,929,No license,4,"['arkit', 'augmented-reality', 'ios', 'objecti..."
56256,Nuke,kean,Image loading system,https://api.github.com/repos/kean/Nuke,2015-03-11T10:53:26Z,2024-02-08T14:13:04Z,2024-02-08T17:33:32Z,12109,7716,7716,2024-02-08T14:13:04Z,Swift,507,MIT License,19,"['cache', 'gif', 'image', 'progressive', 'swif..."
56257,KeychainAccess,kishikawakatsumi,Simple Swift wrapper Keychain works iOS watchO...,https://api.github.com/repos/kishikawakatsumi/...,2014-12-24T22:04:11Z,2024-02-09T09:06:03Z,2024-01-12T10:11:48Z,1577,7705,7705,2024-02-09T09:06:03Z,Swift,790,MIT License,53,"['keychain', 'security', 'touch-id']"
56258,Dozer,Mortennn,Hide menu bar icons macOS,https://api.github.com/repos/Mortennn/Dozer,2018-09-30T11:37:51Z,2024-02-09T16:36:35Z,2023-11-30T13:45:27Z,78954,7690,7690,2024-02-09T16:36:35Z,Swift,251,Mozilla Public License 2.0,86,"['macos', 'minimalistic', 'statusbar', 'utility']"


In [4]:
# delete missing values
df = df.dropna()
# check missing values
df.isnull().sum()

name            0
owner_user      0
description     0
url             0
date_created    0
date_updated    0
date_pushed     0
size            0
stars           0
watchers        0
updated_at      0
language        0
num_forks       0
license         0
open_issues     0
topics          0
dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56041 entries, 0 to 56259
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   name          56041 non-null  object
 1   owner_user    56041 non-null  object
 2   description   56041 non-null  object
 3   url           56041 non-null  object
 4   date_created  56041 non-null  object
 5   date_updated  56041 non-null  object
 6   date_pushed   56041 non-null  object
 7   size          56041 non-null  int64 
 8   stars         56041 non-null  int64 
 9   watchers      56041 non-null  int64 
 10  updated_at    56041 non-null  object
 11  language      56041 non-null  object
 12  num_forks     56041 non-null  int64 
 13  license       56041 non-null  object
 14  open_issues   56041 non-null  int64 
 15  topics        56041 non-null  object
dtypes: int64(5), object(11)
memory usage: 7.3+ MB


# Preprocessing and Feature Engineering

### Encoding

In [6]:
# One-hot encoding of categorical variables -> COMPARE with label encoding after
categorical_columns = ['language', 'license', 'owner_user']
df = pd.get_dummies(df, columns=categorical_columns)
df.head()

,name,description,url,date_created,date_updated,date_pushed,size,stars,watchers,updated_at,...,owner_user_zpmorgan,owner_user_zsh-users,owner_user_zvkemp,owner_user_zvoase,owner_user_zwaldowski,owner_user_zwily,owner_user_zxing,owner_user_zyedidia,owner_user_zygmuntz,owner_user_zzgavin
0,beforerender,Add hook like afterfilter runs action rendered,https://api.github.com/repos/spejman/before_re...,2011-04-15T11:30:07Z,2013-01-01T21:48:40Z,2010-09-06T03:36:34Z,84,1,1,2013-01-01T21:48:40Z,...,0,0,0,0,0,0,0,0,0,0
1,bliptv,Ruby library Bliptv API,https://api.github.com/repos/spejman/bliptv,2009-09-06T01:07:24Z,2012-12-13T01:05:54Z,2009-09-09T08:06:39Z,131,1,1,2012-12-13T01:05:54Z,...,0,0,0,0,0,0,0,0,0,0
2,bn4r,Bayesian networks Ruby,https://api.github.com/repos/spejman/bn4r,2008-06-20T09:54:42Z,2019-08-13T13:27:12Z,2012-07-22T19:10:22Z,180,19,19,2019-08-13T13:27:12Z,...,0,0,0,0,0,0,0,0,0,0
3,cachemoney,WriteThrough Cacheing Library ActiveRecord,https://api.github.com/repos/spejman/cache-money,2011-04-09T09:33:31Z,2013-01-01T20:33:32Z,2011-03-16T22:45:43Z,874,1,1,2013-01-01T20:33:32Z,...,0,0,0,0,0,0,0,0,0,0
4,contacts,universal interface import email contacts prov...,https://api.github.com/repos/spejman/contacts,2009-08-13T11:16:05Z,2012-12-13T00:14:44Z,2009-08-07T19:09:11Z,106,1,1,2012-12-13T00:14:44Z,...,0,0,0,0,0,0,0,0,0,0


### Vectorizing textual data and converting them to pytorch tensors for later model training

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Concatenate 'description' and 'topics' columns for fitting the TF-IDF vectorizer
all_text = pd.concat([df['description'].fillna(''), df['topics'].fillna('')])

# Initializing TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')

# Fitting the vectorizer on the entire text corpus
tfidf_vectorizer.fit(all_text)

# Now, transform each text column separately
tfidf_description = tfidf_vectorizer.transform(df['description'].fillna('')).toarray()
tfidf_topics = tfidf_vectorizer.transform(df['topics'].fillna('')).toarray()

# Converting TF-IDF results to PyTorch tensors
tfidf_description_tensor = torch.tensor(tfidf_description, dtype=torch.float32)
tfidf_topics_tensor = torch.tensor(tfidf_topics, dtype=torch.float32)

### Normalization

In [8]:
from sklearn.preprocessing import MinMaxScaler

# Selecting numerical features for normalization
numerical_features = ['stars', 'num_forks', 'open_issues', 'watchers', 'size']

# Initializing the MinMaxScaler
scaler = MinMaxScaler()

# Applying normalization to the numerical features
df[numerical_features] = scaler.fit_transform(df[numerical_features])

# Converting the normalized data to PyTorch tensors
normalized_features_tensor = torch.tensor(df[numerical_features].values, dtype=torch.float32)

# Displaying the first few rows of the normalized dataframe and the shape of the tensor
df[numerical_features].head(), normalized_features_tensor.shape


(      stars  num_forks  open_issues  watchers      size
 0  0.000003   0.000000          0.0  0.000003  0.000005
 1  0.000003   0.000011          0.0  0.000003  0.000008
 2  0.000050   0.000067          0.0  0.000050  0.000010
 3  0.000003   0.000000          0.0  0.000003  0.000050
 4  0.000003   0.000011          0.0  0.000003  0.000006,
 torch.Size([56041, 5]))

#### Simulated User Preferences

In [9]:
# Creating a simulated user-repo interaction matrix
simulated_users = {'user1': 'language_Python', 'user2': 'language_JavaScript', 'user3': 'language_Java'}

# Initialize the interaction matrix as a NumPy array of zeros
num_rows = df.shape[0]
interaction_matrix_np = np.zeros((num_rows, len(simulated_users)))

# Populate the interaction matrix using NumPy
user_columns = [simulated_users[user] for user in simulated_users]
for i, column in enumerate(user_columns):
    interaction_matrix_np[:, i] = df[column].values

# Convert the NumPy array to a Pandas DataFrame (optional, for verification)
interaction_matrix_df = pd.DataFrame(interaction_matrix_np, columns=simulated_users.keys())

# Convert the interaction matrix to a PyTorch tensor
interaction_matrix_tensor = torch.tensor(interaction_matrix_np, dtype=torch.float32)

In [11]:
from sklearn.model_selection import train_test_split

# Concatenate TF-IDF features for 'description' and 'topics'
tfidf_features_tensor = torch.cat((tfidf_description_tensor, tfidf_topics_tensor), dim=1)

# Concatenate all features (TF-IDF for content, normalized numerical features, and collaborative features)
all_features = torch.cat((tfidf_features_tensor, normalized_features_tensor, interaction_matrix_tensor), dim=1)

# DUMMY TARGET VARIABLE -> THE IDEAL TARGET VARIABLE IS A COSINE SIMILARITY BETWEEN THE USER INTERACTION AND THE REPO -> We don't have this information.
target = torch.randint(0, 2, (all_features.shape[0], 1), dtype=torch.float)

# Splitting the data into training and test sets
features_train, features_test, target_train, target_test = train_test_split(
    all_features, target, test_size=0.2, random_state=42
)


In [12]:
import torch.nn as nn
import torch.nn.functional as F

class HybridRecommendationModel(nn.Module):
    def __init__(self, num_features, num_hidden=128):
        super(HybridRecommendationModel, self).__init__()
        self.fc1 = nn.Linear(num_features, num_hidden)
        self.fc2 = nn.Linear(num_hidden, num_hidden)
        self.output = nn.Linear(num_hidden, 1)
        
        # Weight initialization
        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.xavier_uniform_(self.fc2.weight)
        nn.init.xavier_uniform_(self.output.weight)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = torch.sigmoid(self.output(x))
        return x

# Number of features in your dataset
num_features = features_train.shape[1]

# Instantiate the model
model = HybridRecommendationModel(num_features)

In [13]:
# Converting input tensors to float32
features_train = features_train.float()
features_test = features_test.float()
target_train = target_train.float()
target_test = target_test.float()

In [14]:
# Define a loss function and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

batch_size = 32  # Adjust based on your dataset
# Gradient clipping value
clip_value = 1.0

# Training loop
num_epochs = 70
for epoch in range(num_epochs):
    total_loss = 0.0
    num_batches = 0
    for i in range(0, len(features_train), batch_size):
        batch_features = features_train[i:i + batch_size]
        batch_targets = target_train[i:i + batch_size]

        # Forward pass
        outputs = model(batch_features)
        outputs = torch.clamp(outputs, 0, 1)

        loss = criterion(outputs, batch_targets)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip_value)
 
        optimizer.step()

        total_loss += loss.item()
        num_batches += 1

    average_loss = total_loss / num_batches
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {average_loss:.4f}")



Epoch 1/70, Loss: 0.6932
Epoch 2/70, Loss: 0.6910
Epoch 3/70, Loss: 0.6858
Epoch 4/70, Loss: 0.6772
Epoch 5/70, Loss: 0.6673
Epoch 6/70, Loss: 0.6563
Epoch 7/70, Loss: 0.6435
Epoch 8/70, Loss: 0.6292
Epoch 9/70, Loss: 0.6142
Epoch 10/70, Loss: 0.5984
Epoch 11/70, Loss: 0.5825
Epoch 12/70, Loss: 0.5674
Epoch 13/70, Loss: 0.5535
Epoch 14/70, Loss: 0.5411
Epoch 15/70, Loss: 0.5301
Epoch 16/70, Loss: 0.5205
Epoch 17/70, Loss: 0.5119
Epoch 18/70, Loss: 0.5043
Epoch 19/70, Loss: 0.4976
Epoch 20/70, Loss: 0.4915
Epoch 21/70, Loss: 0.4862
Epoch 22/70, Loss: 0.4814
Epoch 23/70, Loss: 0.4770
Epoch 24/70, Loss: 0.4731
Epoch 25/70, Loss: 0.4697
Epoch 26/70, Loss: 0.4666
Epoch 27/70, Loss: 0.4637
Epoch 28/70, Loss: 0.4612
Epoch 29/70, Loss: 0.4588
Epoch 30/70, Loss: 0.4566
Epoch 31/70, Loss: 0.4546
Epoch 32/70, Loss: 0.4528
Epoch 33/70, Loss: 0.4512
Epoch 34/70, Loss: 0.4497
Epoch 35/70, Loss: 0.4483
Epoch 36/70, Loss: 0.4470
Epoch 37/70, Loss: 0.4459
Epoch 38/70, Loss: 0.4447
Epoch 39/70, Loss: 0.

In [15]:
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    test_outputs = model(features_test)
    test_loss = criterion(test_outputs, target_test)
print(f"Test Loss: {test_loss.item()}")

Test Loss: 1.426355004310608
